In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

In [3]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=64,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)

In [4]:
from allennlp.common.checks import ConfigurationError

In [5]:
USE_GPU = torch.cuda.is_available()

In [6]:
DATA_ROOT = Path("../data") / "jigsaw"

Set random seed manually to replicate results

In [7]:
torch.manual_seed(config.seed)

# Load Data

In [8]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

### Prepare dataset

In [9]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

In [10]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

class JigsawDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token], id: str,
                         labels: np.ndarray) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        if config.testing: df = df.head(1000)
        for i, row in df.iterrows():
            yield self.text_to_instance(
                [Token(x) for x in self.tokenizer(row["comment_text"])],
                row["id"], row[label_cols].values,
            )

### Prepare token handlers

We will use the spacy tokenizer here

In [11]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers.elmo_indexer import ELMoTokenCharactersIndexer

# the token indexer is responsible for mapping tokens to integers
token_indexer = ELMoTokenCharactersIndexer()

def tokenizer(x: str):
    return [w.text for w in
            SpacyWordSplitter(language='en_core_web_sm', 
                              pos_tags=False).split_words(x)[:config.max_seq_len]]

In [12]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [13]:
train_ds, test_ds = (reader.read(DATA_ROOT / fname) for fname in ["train.csv", "test_proced.csv"])
val_ds = None

267it [00:02, 128.71it/s]
251it [00:01, 204.83it/s]


In [14]:
len(train_ds)

267

In [15]:
train_ds[:10]

In [16]:
vars(train_ds[0].fields["tokens"])

{'tokens': [Explanation,
  Why,
  the,
  edits,
  made,
  under,
  my,
  username,
  Hardcore,
  Metallica,
  Fan,
  were,
  reverted,
  ?,
  They,
  were,
  n't,
  vandalisms,
  ,,
  just,
  closure,
  on,
  some,
  GAs,
  after,
  I,
  voted,
  at,
  New,
  York,
  Dolls,
  FAC,
  .,
  And,
  please,
  do,
  n't,
  remove,
  the,
  template,
  from,
  the,
  talk,
  page,
  since,
  I,
  'm,
  retired,
  now.89.205.38.27],
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.elmo_indexer.ELMoTokenCharactersIndexer at 0x1a27f67550>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None}

### Prepare vocabulary

In [17]:
vocab = Vocabulary.from_instances(train_ds, max_vocab_size=config.max_vocab_size)

01/30/2019 14:32:16 - INFO - allennlp.data.vocabulary -   Fitting token dictionary from dataset.
100%|██████████| 267/267 [00:00<00:00, 67560.28it/s]


### Prepare iterator

The iterator is responsible for batching the data and preparing it for input into the model. We'll use the BucketIterator that batches text sequences of smilar lengths together.

In [18]:
from allennlp.data.iterators import BucketIterator

In [19]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("tokens", "num_tokens")],
                         )

We need to tell the iterator how to numericalize the text data. We do this by passing the vocabulary to the iterator. This step is easy to forget so be careful! 

In [20]:
iterator.index_with(vocab)

### Read sample

In [21]:
batch = next(iter(iterator(train_ds)))

In [22]:
batch

{'tokens': {'tokens': tensor([[[259,  74, 260,  ..., 261, 261, 261],
           [259,  71,  74,  ..., 261, 261, 261],
           [259,  74,  85,  ..., 261, 261, 261],
           ...,
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0]],
  
          [[259,  35, 260,  ..., 261, 261, 261],
           [259,  86,  79,  ..., 261, 261, 261],
           [259,  78,  70,  ..., 261, 261, 261],
           ...,
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0]],
  
          [[259,  73, 112,  ..., 261, 261, 261],
           [259, 100,  98,  ..., 261, 261, 261],
           [259, 112, 111,  ..., 261, 261, 261],
           ...,
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0]],
  
          ...,
  
          [[259,  35

In [23]:
batch["tokens"]["tokens"]

tensor([[[259,  74, 260,  ..., 261, 261, 261],
         [259,  71,  74,  ..., 261, 261, 261],
         [259,  74,  85,  ..., 261, 261, 261],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[259,  35, 260,  ..., 261, 261, 261],
         [259,  86,  79,  ..., 261, 261, 261],
         [259,  78,  70,  ..., 261, 261, 261],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[259,  73, 112,  ..., 261, 261, 261],
         [259, 100,  98,  ..., 261, 261, 261],
         [259, 112, 111,  ..., 261, 261, 261],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        ...,

        [[259,  35, 260,  ..., 261, 261, 261],
         [259,  50,  51,  ..., 261, 261, 261]

In [24]:
batch["tokens"]["tokens"].shape

torch.Size([64, 46, 50])

# Prepare Model

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim

In [26]:
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder

class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

### Prepare embeddings

In [27]:
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import ElmoTokenEmbedder

options_file = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_options.json'
weight_file = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5'

elmo_embedder = ElmoTokenEmbedder(options_file, weight_file)
word_embeddings = BasicTextFieldEmbedder({"tokens": elmo_embedder})

01/30/2019 14:32:17 - INFO - allennlp.modules.elmo -   Initializing ELMo


In [28]:
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
encoder: Seq2VecEncoder = PytorchSeq2VecWrapper(nn.LSTM(word_embeddings.get_output_dim(), config.hidden_sz, bidirectional=True, batch_first=True))

Notice how simple and modular the code for initializing the model is. All the complexity is delegated to each component.

In [29]:
model = BaselineModel(
    word_embeddings, 
    encoder, 
)

In [30]:
if USE_GPU: model.cuda()
else: model

# Basic sanity checks

In [31]:
batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

In [32]:
tokens = batch["tokens"]
labels = batch

In [33]:
tokens

{'tokens': tensor([[[259,  74, 260,  ..., 261, 261, 261],
          [259,  71,  74,  ..., 261, 261, 261],
          [259,  74,  85,  ..., 261, 261, 261],
          ...,
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0]],
 
         [[259,  35, 260,  ..., 261, 261, 261],
          [259,  86,  79,  ..., 261, 261, 261],
          [259,  78,  70,  ..., 261, 261, 261],
          ...,
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0]],
 
         [[259,  73, 112,  ..., 261, 261, 261],
          [259, 100,  98,  ..., 261, 261, 261],
          [259, 112, 111,  ..., 261, 261, 261],
          ...,
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0]],
 
         ...,
 
         [[259,  35, 260,  ..., 261, 261, 261],
        

In [34]:
mask = get_text_field_mask(tokens)
mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 1]])

In [35]:
embeddings = model.word_embeddings(tokens)
state = model.encoder(embeddings, mask)
class_logits = model.projection(state)
class_logits

tensor([[ 2.2969e-02,  3.3753e-02,  1.2022e-01,  1.4575e-02,  2.2457e-01,
         -2.0415e-01],
        [-1.9613e-01, -1.7912e-02,  2.9364e-02,  1.1163e-01, -1.2791e-02,
         -2.8244e-02],
        [-1.0925e-01,  1.2709e-01,  7.7944e-02,  1.3847e-01,  7.8998e-03,
         -2.1527e-01],
        [-1.7458e-02, -1.0176e-02,  7.8199e-02, -1.0448e-01, -4.1375e-02,
         -7.5978e-02],
        [ 1.0936e-01, -8.1456e-03, -1.2194e-02,  1.8560e-02,  3.4497e-02,
         -8.4653e-02],
        [-5.1728e-02,  6.2288e-02, -1.2658e-02, -1.7544e-01,  1.1073e-01,
         -9.4841e-02],
        [ 1.1473e-02,  3.5975e-02,  1.0825e-01, -9.4212e-02,  1.3700e-01,
         -1.3121e-01],
        [-8.2636e-02, -3.1713e-02,  8.1382e-02,  3.3022e-02,  1.6678e-01,
         -3.2406e-02],
        [ 1.0901e-01,  1.0439e-01, -2.2460e-02, -9.4008e-02,  2.2116e-01,
         -1.8600e-01],
        [-1.5247e-01,  1.2574e-01,  2.6344e-01,  7.5079e-02,  8.9553e-02,
         -8.4772e-02],
        [-3.5631e-02, -1.0768e

In [36]:
model(**batch)

{'class_logits': tensor([[-3.1033e-02,  2.3331e-02,  1.2371e-01, -1.2836e-01,  1.4709e-01,
          -1.7873e-01],
         [-1.3789e-01,  2.4056e-02, -3.4560e-02, -6.0254e-02, -7.3038e-03,
           2.7147e-03],
         [-1.1190e-01, -6.4721e-02, -2.6389e-01,  2.0364e-01,  6.4780e-02,
          -3.8040e-02],
         [-3.3587e-02,  3.5838e-02,  4.4031e-02, -6.5696e-03, -2.8547e-02,
          -1.2327e-01],
         [ 3.7575e-02,  3.3898e-02,  8.6971e-02,  1.8634e-02, -2.6618e-02,
          -1.6192e-01],
         [-8.1112e-02,  9.8188e-02,  1.6625e-01, -2.1964e-01,  9.3888e-02,
          -1.3452e-01],
         [ 3.0536e-02, -2.4114e-02,  8.0818e-03, -8.3188e-02,  2.1043e-01,
          -1.0255e-01],
         [-1.8540e-02,  5.7744e-02, -7.2206e-04, -4.2356e-02,  1.9489e-01,
          -1.4831e-01],
         [-6.0753e-02,  1.3936e-02,  1.2929e-01,  5.3706e-02,  1.9803e-01,
          -4.6742e-02],
         [-1.2123e-01, -2.9820e-02,  1.8571e-01, -2.7066e-02,  6.7571e-02,
          -1.8325e

In [37]:
loss = model(**batch)["loss"]

In [38]:
loss

tensor(0.6979, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [39]:
loss.backward()

In [40]:
[x.grad for x in list(model.encoder.parameters())]

[tensor([[ 2.1172e-06,  5.4381e-05,  5.5318e-06,  ..., -3.8330e-06,
           5.6735e-05,  2.6381e-05],
         [-1.0820e-04,  5.1226e-05, -3.8212e-05,  ..., -8.9566e-05,
           1.4772e-04, -2.3325e-05],
         [-1.9074e-04,  1.1211e-04,  1.4580e-05,  ...,  3.1020e-05,
           2.5292e-04,  9.4283e-05],
         ...,
         [ 1.8176e-04, -9.9353e-05,  7.2918e-05,  ..., -3.9730e-05,
          -2.6642e-04, -3.5973e-06],
         [-3.0664e-05,  9.0954e-05,  6.9829e-05,  ..., -3.1430e-05,
           5.4184e-05, -2.0345e-05],
         [ 1.6058e-04, -8.0744e-05,  1.4813e-05,  ..., -1.0257e-04,
          -2.5983e-04,  1.5409e-04]]),
 tensor([[-2.6208e-05, -2.2792e-07,  5.7287e-06,  ..., -2.4225e-05,
           5.4475e-06,  1.3612e-05],
         [-1.4566e-05,  1.1434e-05, -2.7109e-05,  ...,  5.0374e-05,
          -1.8014e-06,  9.5703e-06],
         [ 1.7004e-05,  2.4735e-05, -6.0654e-05,  ...,  1.4248e-05,
          -1.1867e-05,  4.6304e-05],
         ...,
         [ 5.8953e-06, -5

# Train

In [41]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [42]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [43]:
metrics = trainer.train()

01/30/2019 14:32:43 - INFO - allennlp.training.trainer -   Beginning training.
01/30/2019 14:32:43 - INFO - allennlp.training.trainer -   Epoch 0/1
01/30/2019 14:32:43 - INFO - allennlp.training.trainer -   Peak CPU memory usage MB: 1880.24832
01/30/2019 14:32:43 - INFO - allennlp.training.trainer -   Training
loss: 0.6841 ||: 100%|██████████| 5/5 [00:37<00:00,  6.60s/it]
01/30/2019 14:33:21 - INFO - allennlp.training.trainer -                     Training |  Validation
01/30/2019 14:33:21 - INFO - allennlp.training.trainer -   loss          |     0.684  |       N/A
01/30/2019 14:33:21 - INFO - allennlp.training.trainer -   cpu_memory_MB |  1880.248  |       N/A
01/30/2019 14:33:21 - INFO - allennlp.training.trainer -   Epoch duration: 00:00:37
01/30/2019 14:33:21 - INFO - allennlp.training.trainer -   Estimated training time remaining: 0:00:37
01/30/2019 14:33:21 - INFO - allennlp.training.trainer -   Epoch 1/1
01/30/2019 14:33:21 - INFO - allennlp.training.trainer -   Peak CPU memory